# COGS 118B - Final Project

# Spaceship Game: Space the Final Frontier for RL

## Group members

- Fabian Tellez (A18095006)
- Brandon Chu (A17024601)


# Abstract 
 

 Our goal is to test three different reinforced learning techniques and gauge their results against each other. The three reinforced learning techniques we are applying are DP's Approximate Value (DP), Monte Carlo (MC), and Temporal Difference's Q-Learning (TD). We will attempt to find the most optimal path in a 3D maze, which will include slip events (portals), moving obstacles, and mines. We will use Python to simulate 4000 iterations for MC and Q-learning and up to 60 iterations for Dynamic Programming (DP). Key findings are: DP results in a policy that may differ from model-free methods due to the moving obstacles<a name="lorenz"></a>[<sup>[1]</sup>](#Footnotes), MC and TD most of the time, start off using the same set of actions<a name="lorenz"></a>[<sup>[2]</sup>](#Footnotes), and the random teleports and moving obstacles reduce the achievable returns<a name="lorenz"></a>[<sup>[3]</sup>](#Footnotes). We will measure the efficiency by using the policy choices and the estimated value at sample states. This will highlight the pros and cons and trade-offs between model-based and model-free environments.      

# Background

Reinforncement Learning is a technique that focuses on an agent learning behaviors through trial and error. This trial and error results in points being rewarded for "correct" behaviors and points taken away for "incorrect" behaviors. The more the correct choice is made, the more reward points the agent accumulates, reinforcing the actions we want it to take<a name="lorenz"></a>[<sup>[1]</sup>](#Footnotes)<a name="lorenz"></a>[<sup>[2]</sup>](#Footnotes). Traditionally, reinforced learning used 2D worlds or static Markov Decision processes. This only helps so much since real-world applications tend to be nonstatic. In order to mimic more of a real-world environment, we are making the grid less static by adding moving hazards (representing children running across the street) and shifting transitions and portals in order to introduce some randomness. Dynamic programming usually requires a more stable model versus Monte Carlo and Temporal Difference, which are learned from their experience going through the model<a name="lorenz"></a>[<sup>[1]</sup>](#Footnotes)<a name="lorenz"></a>[<sup>[3]</sup>](#Footnotes). Therefor, we believe that in a 3D environment with moving obstacles, slip events, and model-free algorithms like Monte Carlo and Q-Learning (TD) will learn and adapt faster to the environment and changes than a model-based algorithm like Value Iteration (DP), which will result in better returns and a more stable policy the more iterations there are. 

# Problem Statement

Our problem will involve creating a 3D grid environment with x,y, and z as coordinates, with each variable being able to be between 0 and 9. 
- States: It will include 1000 states as x,y,z -> (10x10x10)
- Actions: There will be six possible actions that are derived from +x/-x, +y,-y / +z,-z
- Obstacles: There will be 15 random blocked cells which the agent can not enter
- Mines: 5 random cells that produce -10 reward points and end the iteration. 
- Slip Probability: 0.2 change after an action to "teleport" randomly.
- Moving Obstacles: Every step that the agent takes will move obstacles in the Z direction, and when it's already at the top level, it will reset to the bottom. (Obstacles shift in +z direction, wrapping around)
- Rewards: each step taken equates to -0.1 pt, reaching goal equates to +1 pt, and -10 pts for hitting a mine. 
- End: The experiment ends when the goal is reached or when the agent steps on a mine. 

- How to Quantify: The agent's return will be the sum of reward points until the iteration ends. 
- What We Measure: We track the cumulated points (learned value) as well as the policy from the start.
- Repeatable? Yes, it is. Since we used a random maze generation that generates random seeds, the same result can be repeated in the future by using the same seed to generate the maze in question. 

We will generate a large dynamic 3D grid to test how well DP (approx Value Iteration), Monte Carlo and Q-Learning (TD) traverse a partially stochastic environement. 
 

# Data

We did not have to obtain or clean any data since our "data" or environment is internally generated by our 3D grid. Instead our data points will be
- randomly placing obstacles and mines
- each state which is a cordinate of x,y,z
- observational transitions of s(current state), a(action), r(reward), s'(next state)


# Proposed Solution

 
1. **Approximate Value Iteration (DP):** The solution results from repeated value iteration sweeps to create a "big list" of how good each cell is. This will be our V(s). The problem arises from the moving obstacles. Since the obstacles move with each corresponding agent's step, it becomes difficult to accurately look at all possible next states and how likely they are. The workaround for this is 
- At each cell (s) and each action (a), the agent tries a move 20 times and calculates where it lands as well as the reward points gained. 
- The 20 outcomes are then averaged to guess the next state result:$$(r_i + \gamma\,V(ns_i))$$


- The best choice is then picked and the V(s) is updated to reflect the best average:
$$
V(s) \leftarrow \max_{a} \frac{1}{N} \sum_{i=1}^{N} 
\Bigl[r_i + \gamma\,V\bigl(ns_i\bigr)\Bigr].
$$
- The process is then repeated for all remaining cells or until the changes start converging to a really small number or threshold. 

2. **Monte Carlo (MC)** 
The solution results from from being On-Policy with $\epsilon$ to choosing the next action:
$$
Q(s,a) \leftarrow Q(s,a) + \alpha \bigl[G - Q(s,a)\bigr]
$$
- This means that the agent will most of the time (1-$\epsilon$) pick the action it deems is best. 
- During the 4000 iterations, the states, actions, and rewards (s, a,r) are collected at each step. 
- When the iteration ends, the returns are computed and used to update Q(s, a) to see how "good" or "bad" the agent did. This is done by averaging the returns across all cells visited. 
- The policy or action we pick at any time is the action with the highest average score with probability (1-$\epsilon$) and random otherwise. $$\argmax_a Q(s,a)$$ 

3. **Temporal Difference (TD) - Q-Learning:** The solution is similar to Monte Carlo, but the difference is that Q-values are updated after each step. 
$$\begin{aligned}Q(s,a) &\;\leftarrow\; Q(s,a) \;+\; \alpha \Bigl[\,r \;+\;\gamma \,\max_{a'}\,Q\bigl(s',\,a'\bigr) \;-\; Q(s,a)\Bigr],\\[1em]\alpha &\;=\; 0.1,\quad \epsilon \;=\; 0.15\end{aligned}$$
- The final value iteration policy is used as a DP baseline despite the moving obstacles. We are able to do this by assuming that the grid is stationary for a split second, which is just long enough to run value iteration, and then compare the results from MC and TD to DP.  



# Evaluation Metrics

The evaluation metrics we are using will tell us how well each algorithm handles the nonstatic grid environment. We accomplish this by monitoring which action is taken first, how the agent gauges key states, how soon the agent settles down, and how often it succeeds:

1. **Start-State Action:** Which is the first action taken from (0,0,0). It's asking, "Which direction does the agent go when it starts?"    

2. **Value of Key States:** We check how each algorithm scored in certain key coordinates like the start (0,0,0) or somewhere in the middle of the grid (1,1,1). This value will tell us if the agent deems the spot is safe or dangerous. It works as a reference point to see how well the algorithim understands the environment.

3. **Convergence:** 
    - For **DP,** we check to see how many loops (sweeps) happen before the changes converge to a small number. 
    - For **MC & TD,** we calculate how many episodes the agent needs before either the plan starts changing too much or the reward points stop changing. 

4. **Success Rate:** The success rate is is how often the agent reached the goal compared to how many times it stepped on a mine or had to give up. 

# Results

**Main Point:** 
    The policy from Value Iteration (DP) is usually different at the start compared to that of Monte Carlo and Q-Learning. This reflects how DP acts as if it knows the map (model), but since our grid has moving obstacles, it's constantly changing, so the plan ends up different from what actually happens when the agent begins to move. This reflects the disconnect between the environment sampling and the agent's real-time experience. Both **MC and TD** usually picked the same starting move in our final run $(z+1)$. That means that MC and TD both take into consideration the changes in the world and determined that going Z first is safer. **MC and DP** showed learning, with **MC** showing a positive slope and **DP** showing convergence. However, our **TD/Q-learning** algorithm did not show any learning.

![Image description](dp_learning.png)
![Image description](monte_carlo_learning.png)
![Image description](q_learning.png)

**SubPoint - DP (Approx Value Interation):**
    Since the environment the agent is in is constantly changing, the sampling DP does is only an approximation. Despite the changing environment, DP was able to find a small positive value (~0.10) at (0,0,0) which results in the agent deciding to move positive x from there. 

**SubPoint - Monte Carlo & TD**
    Since both MC and TD analyze the world they both realized that going into the positive z direction was the best move. This shows that the agents learned from each step so when the environment changed so did their appreach. As the enviroment changes they adapt <a name="lorenz"></a>[<sup>[3]</sup>](#Footnotes)<a name="lorenz"></a>[<sup>[4]</sup>](#Footnotes). The final Q values near the goal reflect high risk from mines in certain columns (z direction). TD had the shortest episode length, meaning that it took the least time to explore the state space and had the fastest convergence to a terminal state.


![Image description](episode_length.png)

**SubPoint - Further Observations**
    - **Eligibility Traces** would allow us to speed up the learning because it helps share new info faster back to earlier states <a name="lorenz"></a>[<sup>[4]</sup>](#Footnotes).We would do this by more quickly assigning a positive or negative results to the actions that led to reward or penalty<a name="lorenz"></a>[<sup>[4]</sup>](#Footnotes).  
    - If the grid were larger, for example a (20x20x20), then the DP would fall off even faster compared to MC and TD because the costs of sampling grow significally making MC and TD the more practical choice. 

# Discussion

### Interpreting the result

Model free algorithims like MC and TD work b etter when in a non static environment. This happens because model free algorithims rely on their experience in the environment rather than knowing than mapping out the environment. 

If the slip probabilty is set too high than the final returns are greatly lowered because the agent keeps teleporting to mines resulting in the end of that episode. 


### Limitations
Since we used tabular Q-tables insread of an advanced function approximation because of the size of the environemnt. However, a nueral network appreach will work better for much larger environments or continuous spaces<a name="lorenz"></a>[<sup>[5]</sup>](#Footnotes).

Another limitation was not using eligibility traces since using them would combine the pros of multi step back ups and the speed of TD. 

In order to take advantage of a fully Markov property we could have used "time as state" in order to better deal with the moving obstacles. 

### Future work

1. Use a TD($\lambda$) approach or n step returns in order to gauge if partial backups speed up learning in the moving 3D Grid <a name="lorenz"></a>[<sup>[4]</sup>](#Footnotes).

2. Possibly use Neural Network approximators for the x,y, and z states if the grid space becomes very big <a name="lorenz"></a>[<sup>[5]</sup>](#Footnotes). For example, if the Grid becomes 50x50x50 or if we only have partial visibility of the agent.

3. Keep trying multiple random seed generations to measure how the policy's actions react to different obstacles and mine placements.  



### Ethics & Privacy

No human user data is involved in this project so there is no ethical or privacy concerns. 

### Conclusion

We introduced a non static 3D grid environment with random slips, moving obstacles, and mines. After the environment has been created we compare how Dynamic Programming (approx Value Iteration), Monte Carlo, and Temperal Difference performed against each other. 

The results were...
1. DP picked a different initial action than MC or TD which highlights how short comings of model based algorithims and how TD and MC both learn from their environments. 

2. MC and TD both discovered a similar route, probably because it learned it was the safest move the avoid a mine or teleport.

3. Slips and obstacles that impede movement interfere with the maximum return which results in a modest value for the start state. 

In conclusion, these experiments help confirm how a model-free algorithm can cope better and adapt to changing environments, while the DP approach relies more heavily on a static environment, so its sampling can be more weighted. 

# Footnotes
<a name="Sutton1"></a>1. Sutton, R.S., & Barto. A.G. "Dynamic Programming" Reinforcement Learning: An Introduction. *MIT Press*, 2018. PDF file: 05_DynamicProgramming.pdf. <br> 

<a name="Sutton2"></a>2. Sutton, R.S., & Barto, A.G. "Monte Carlo Methods" Reinforcement Learning: An Introduction. MIT Press, 2018. PDF file: 06_MonteCarlo.pdf. <br>

<a name="Sutton3"></a>3. Sutton, R.S., & Barto, A.G. "Chapter 6: Temporal Difference Learning" Reinforcement Learning: An Introduction. MIT Press, 2018.PDF file: 07_01_temporalDifference.pdf.<br>

<a name="Sutton4"></a>4. Sutton, R.S., & Barto, A.G. "Eligibility Traces and n-Step TD" Reinforcement Learning: An Introduction. MIT Press, 2018. PDF file: 08_01_eligibility_td(n).pdf.<br>

<a name="Sutton5"></a>5. Sutton, R.S., & Barto, A.G. "Neural Network Approximations" Reinforcement Learning: An Introduction. MIT Press, 2018. PDF file: 08_02_neuralnetworks.pdf<br>
